# Example Problem -- Cerberus

We are given three points $(x_1, y_1), (x_2, y_2), \textrm{ and } (x_3, y_3)$ and we need to find the coordinates of the point $(x_c, y_c)$ such that the line segments joining this point to the three given points will make angles of $120^\circ$ with each other. In the image below, we are given the points $A, B, \textrm{ and } C$ and we need to determine the point $O$ such that the angles $\angle AOB = \angle BOC = \angle COA = 120^\circ$.

![The Cerberus Problem](files/cerberus.png)

We can use the cosine formula to put constraints on the point $O$:
- $\|AB\|^2 = \|AO\|^2 + \|BO\|^2 - 2\|AO\|\|BO\|\cos 120^\circ$
- $\|AC\|^2 = \|AO\|^2 + \|CO\|^2 - 2\|AO\|\|CO\|\cos 120^\circ$

Note that putting a third constraint would be redundant. Depending on the problem, it may or it may not be a good idea.

We can write each constraint as a function of the coordinates of the point $O$:

- $f_1(x_c, y_c) = \|AO\|^2 + \|BO\|^2 - 2\|AO\|\|BO\|\cos 120^\circ - \|AB\|^2$
- $f_2(x_c, y_c) = \|AO\|^2 + \|CO\|^2 - 2\|AO\|\|CO\|\cos 120^\circ - \|AC\|^2$

Our problem is finding the simultaneous root of the functions $f_1(x_c, y_c)$ and $f_2(x_c, y_x)$. So the input is two dimensional and the output is also two dimensional. In this case the Newton-Raphson iterations are given by
$$ \mathtt{J} \mathbf{\Delta x}_i = - \mathbf{f}(\mathbf{x}_i) $$
where $\mathbf{x} = (x_i, y_i)^\top$ is the coordinate vector of the point $O$, $\mathbf{f}(\mathbf{x}_i)=(f_1(x_i, y_i), f_2(x_i, y_i))^\top$ is the value vector of the function we want to find the root of, and finally
$$ \mathtt{J} = \left[\begin{array}{cc}\frac{\partial f_1}{\partial x}&\frac{\partial f_1}{\partial y}\\\frac{\partial f_2}{\partial x}&\frac{\partial f_2}{\partial y}\end{array}\right]$$
is the Jacobian matrix of the function partial derivatives.

In this homework you will write the necessary Python functions to compute the function values and the Jacobian.

In [ ]:
import math
import numpy as np
from sympy import *

We can define a new function that calculates $f_i(x_c, y_c), i=1,2$. Complete the function below so `f1` and `f2` has the correct values as defined at the beginning. \emph{Hint:} You can check the `angles` function below for a computation of lengths.

In [ ]:
def f_i(xc, yc, corners):
    x1 = corners[0][0]
    y1 = corners[0][1]
    x2 = corners[1][0]
    y2 = corners[1][1]
    x3 = corners[2][0]
    y3 = corners[2][1]
    f1 = # FILL IN
    f2 = # FILL IN
    return np.array([[f1], [f2]])

Now we need to calculate the derivatives of $f_1$ and $f_2$ w.r.t. $x_c$ and $y_c$. We will use the `sympy` module to do that. The code below is almost complete. You just need to assign `f1` and `f2` to the correct values that you have written above and then run `diff(function, var)` to compute the derivative of a `function` w.r.t. a `variable`. Compute the derivatives $\frac{\partial f_1}{\partial x}, \frac{\partial f_1}{\partial y}, \frac{\partial f_2}{\partial x}, \textrm{ and } \frac{\partial f_2}{\partial y}$.

In [ ]:
x1, x2, x3, xc, y1, y2, y3, yc = symbols('x1 x2 x3 xc y1 y2 y3 yc')
f1 = # FILL IN EXACTLY AS ABOVE
f2 = # FILL IN EXACTLY AS ABOVE
df1_dx = # FILL IN WITH A CALL TO DIFF 
df1_dy = # FILL IN WITH A CALL TO DIFF
df2_dx = # FILL IN WITH A CALL TO DIFF
df2_dy = # FILL IN WITH A CALL TO DIFF
print(f'df1_dx = {df1_dx}')
print(f'df1_dy = {df1_dy}')
print(f'df2_dx = {df2_dx}')
print(f'df2_dy = {df2_dy}')

We need a function to compute the derivative of $f_i$ w.r.t. $x_c$ and $y_c$. Fill in the function below with the output of `sympy`.

In [ ]:
def jacobian(xc, yc, corners):
    x1 = corners[0][0]
    y1 = corners[0][1]
    x2 = corners[1][0]
    y2 = corners[1][1]
    x3 = corners[2][0]
    y3 = corners[2][1]
    df1_dx = # FILL IN WITH THE OUTPUT FROM SYMPY
    df1_dy = # FILL IN WITH THE OUTPUT FROM SYMPY
    df2_dx = # FILL IN WITH THE OUTPUT FROM SYMPY
    df2_dy = # FILL IN WITH THE OUTPUT FROM SYMPY
    J = np.array([[df1_dx, df1_dy], [df2_dx, df2_dy]])
    return J

Later on, we will also learn how to compute the derivatives numerically. To compute the analytic derivative you can use symbolic math software such as the `sympy` module for Python. Now we define a function for a single iteration of the Newton-Raphson algorithm for convinience:

In [ ]:
def iter_analytic(xc, yc, corners):
    fi = f_i(xc, yc, corners)
    J = jacobian(xc, yc, corners)
    xs = np.linalg.solve(J, -fi) # We solve the system of equations for the step xs 
    return xc + xs[0][0], yc + xs[1][0]

We can also measure the angles to better see the quality of the current solution.

In [ ]:
def angles(xc, yc, corners):
    x1 = corners[0][0]
    y1 = corners[0][1]
    x2 = corners[1][0]
    y2 = corners[1][1]
    x3 = corners[2][0]
    y3 = corners[2][1]
    lc1_sqr = (yc-y1)**2 + (xc-x1)**2
    lc3_sqr = (yc-y3)**2 + (xc-x3)**2
    l13_sqr = (y1-y3)**2 + (x1-x3)**2
    lc1lc3 = lc1_sqr**0.5 * lc3_sqr**0.5
    costheta0 = (l13_sqr - lc1_sqr - lc3_sqr) / (-2.0*lc1lc3)

    lc2_sqr = (yc-y2)**2 + (xc-x2)**2
    l12_sqr = (y1-y2)**2 + (x1-x2)**2
    lc1lc2 = lc1_sqr**0.5 * lc2_sqr**0.5
    costheta1 = (l12_sqr - lc1_sqr - lc2_sqr) / (-2.0*lc1lc2)

    return math.acos(costheta0), math.acos(costheta1)

With the definitions above we are ready to solve the problem. Make sure the cell below runs without error. The solution should be found in around 5-6 iterations.

In [ ]:
x1 = 193.9572
y1 = 380.0689
x2 = 388.4872
y2 = 380.0689
x3 = 324.7872
y3 = 257.0696
corners = ((x1, y1), (x2, y2), (x3, y3))

# We initialize the algorithm at the geometric mean of the corners
xi = (x1+x2+x3)/3.0
yi = (y1+y2+y3)/3.0

fi = f_i(xi, yi, corners)
print(f'Initial xc = {xi},yc = {yi}: f1 = {fi[0]}, f2 = {fi[1]}')
theta = angles(xi, yi, corners)
print(f'Angles (degrees) = |AOC| = {theta[0]*180.0/math.pi}, |AOB| = {theta[1]*180.0/math.pi}')

param_tol = 1e-8
max_iter = 1000
for i in range(max_iter):
    xn, yn = iter_analytic(xi, yi, corners)
    dx = xn-xi
    dy = yn-yi
    xi, yi = xn, yn
    print(f'Iter {i+1}: xi = {xi:.8f}, yi = {yi:.8f}, dx = {dx:.2g}, dy = {dy:.2g}')
    if abs(dx) < param_tol and abs(dy) < param_tol:
        break


We can check the quality of the final solution.

In [ ]:
print('Final Solution: xc = {},yc = {}'.format(xi, yi))
theta = angles(xi, yi, corners)
print(f'Angles (degrees) = |AOC| = {theta[0]*180.0/math.pi}, |AOB| = {theta[1]*180.0/math.pi}')